## Workshop 4: Problem Feasibility and Restoration Problem

### 1. Installing and Importing Packages 

We first need to pull in all the packages we will be using. Pyomo is a Python-based, open-source optimization modelling language with a diverse set of optimization capabilities. For more information, see the Pyomo [documentation](https://pyomo.readthedocs.io/en/stable/).

In [1]:
# Only run once at the start
!pip install -q pyomo
!pip install -q matplotlib

In [2]:
import matplotlib.pyplot as plt
from pyomo.environ import *
import numpy as np
from ipywidgets import FloatSlider, interact
import platform

# Ipopt solver setup for Windows or Linux
def setup_solver():
    os_name = platform.system()
    if os_name == "Windows":
        return "solver/ipopt.exe"
    elif os_name == "Linux":
        !chmod +x "solver/ipopt"
        return "solver/ipopt"


solver_executable = setup_solver()

In [58]:
# Create a model
model = ConcreteModel()

# Define variables
model.x = Var(bounds=(0, 1))
model.y = Var(bounds=(0, 1))

# Define the objective function
model.obj = Objective(expr=model.x + model.y, sense=maximize)

# Define constraints
model.constr1 = Constraint(expr=model.x + model.y >= 1)                  # Inequality constraint
model.constr2 = Constraint(expr=model.x + model.y >= 0.9999)             # Tight inequality constraint

# Create a solver with a custom executable
solver = SolverFactory('ipopt', executable=solver_executable)

# Solve the model
results = solver.solve(model, tee=True)

# Display results
model.display()

Ipopt 3.9.1: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Common Public License (CPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

NOTE: You are using Ipopt by default with the MUMPS linear solver.
      Other linear solvers might be more efficient (see Ipopt documentation).


This is Ipopt version 3.9.1, running with linear solver mumps.

Number of nonzeros in equality constraint Jacobian...:        2
Number of nonzeros in inequality constraint Jacobian.:        2
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        2
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        2
   

In [ ]:
# Create a model
model = ConcreteModel()

# Define variables
model.x = Var(bounds=(0, 1))
model.y = Var(bounds=(0, 1))

# Define the objective function
model.obj = Objective(expr=model.x + model.y, sense=maximize)

# Define constraints
model.constr1 = Constraint(expr=model.x + model.y <= 1)                  # Inequality constraint
model.constr2 = Constraint(expr=model.x + model.y >= 0.9999)             # Tight inequality constraint

# Create a solver with a custom executable
solver = SolverFactory('ipopt', executable=solver_executable)

# Solve the model
results = solver.solve(model, tee=True)

# Display results
model.display()

In [47]:
# Importing the necessary libraries
from pyomo.environ import *

# Step 1: Define the optimization model
model = ConcreteModel()

# Define decision variables
model.x1 = Var(initialize=1.0)
model.x2 = Var(initialize=1.0)

# Define a simple objective function: Minimize distance from (1, 2)
model.obj = Objective(expr=(model.x1 - 1)**2 + (model.x2 - 2)**2, sense=minimize)

# Define a very steep, ill-conditioned nonlinear constraint with sharp transitions
model.constr1 = Constraint(expr=1e12 * (model.x1**6 - 3*model.x1**2) + 1e12 * (model.x2**3 - model.x2) == 0)

# Step 2: Solve using IPOPT with the user-specified solver executable
solver = SolverFactory('ipopt', executable=solver_executable)
results = solver.solve(model, tee=True)

# Print the solution status and values
print(f"IPOPT status: {results.solver.termination_condition}")
print(f"x1 = {value(model.x1)}, x2 = {value(model.x2)}")


Ipopt 3.9.1: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Common Public License (CPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

NOTE: You are using Ipopt by default with the MUMPS linear solver.
      Other linear solvers might be more efficient (see Ipopt documentation).


This is Ipopt version 3.9.1, running with linear solver mumps.

Number of nonzeros in equality constraint Jacobian...:        2
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        2

Total number of variables............................:        2
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
   

ValueError: Cannot load a SolverResults object with bad status: error

In [43]:
# Step 1: Define the restoration optimization model
rest_model = ConcreteModel()

# Define decision variables
rest_model.x1 = Var(initialize=1.0)
rest_model.x2 = Var(initialize=1.0)
rest_model.epsilon = Var(within=NonNegativeReals, initialize=0.0)  # Epsilon to measure constraint violation

# Step 2: Define the objective function to minimize epsilon
rest_model.obj = Objective(expr=(1e6 * (rest_model.x1**6 - 3 * rest_model.x1**2) + 1e6 * (rest_model.x2**3 - rest_model.x2) + rest_model.epsilon), sense=minimize)

# Step 3: Solve the restoration model using IPOPT with the user-specified solver executable
solver = SolverFactory('ipopt', executable=solver_executable)
results = solver.solve(rest_model, tee=True)

# Print the solution status and values
print(f"Restoration Model IPOPT status: {results.solver.termination_condition}")
print(f"x1 = {value(rest_model.x1)}, x2 = {value(rest_model.x2)}, epsilon = {value(rest_model.epsilon)}")


Ipopt 3.9.1: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Common Public License (CPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

NOTE: You are using Ipopt by default with the MUMPS linear solver.
      Other linear solvers might be more efficient (see Ipopt documentation).


This is Ipopt version 3.9.1, running with linear solver mumps.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        2

Total number of variables............................:        3
                     variables with only lower bounds:        1
                variables with lower and upper bounds:        0
   

In [42]:
# Importing the necessary libraries
from pyomo.environ import *

# Step 1: Define the optimization model
model = ConcreteModel()

# Define decision variables
model.x1 = Var(initialize=1.0)
model.x2 = Var(initialize=0.5773502691896257)

# Define a simple objective function: Minimize distance from (1, 2)
model.obj = Objective(expr=(model.x1 - 1)**2 + (model.x2 - 2)**2, sense=minimize)

# Define a very steep, ill-conditioned nonlinear constraint with sharp transitions
model.constr1 = Constraint(expr=1e12 * (model.x1**6 - 3*model.x1**2) + 1e12 * (model.x2**3 - model.x2) == 0)

# Step 2: Solve using IPOPT with the user-specified solver executable
solver = SolverFactory('ipopt', executable=solver_executable)
results = solver.solve(model, tee=True)

# Print the solution status and values
print(f"IPOPT status: {results.solver.termination_condition}")
print(f"x1 = {value(model.x1)}, x2 = {value(model.x2)}")


Ipopt 3.9.1: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Common Public License (CPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

NOTE: You are using Ipopt by default with the MUMPS linear solver.
      Other linear solvers might be more efficient (see Ipopt documentation).


This is Ipopt version 3.9.1, running with linear solver mumps.

Number of nonzeros in equality constraint Jacobian...:        2
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        2

Total number of variables............................:        2
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
   